# Gradient-Effected Object Recognition Gauge for hive Entrances (GEORGE)
Neural-net-powered honeybee hive-mounted pollen, varroa, and wasp counter

## Imports

In [1]:
import os
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt
import random
import glob
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
import pandas as pd
import keras
import keras_cv
from keras_cv import bounding_box
import os
#from keras_cv import visualization
import GEORGE_Library as GEORGE
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("Done")

Done


In [2]:
category_index = {1: {'id': 1, 'name': 'regular'}, 2: {'id': 2, 'name': 'pollen'}, 3: {'id': 3, 'name': 'varroa'}, 4: {'id': 4, 'name': 'wasps'}}
train_augmented_fp = "Dataset/TrainAugmented"
val_augmented_fp = "Dataset/ValAugmented"
IMAGE_WIDTH = 75
IMAGE_HEIGHT = 150
NUM_IMAGES = None
model_directory = 'C:/Users/khard/Documents/GitHub/jupyter/PollenCounter/'
model_name = 'non_extract_non_fill_sub_model'
model_dest = os.path.join(os.sep, model_directory, model_name)
tflite_save = False

# The `label_id_offset` shifts all classes so the model receives one-hot labels where non-background classes start counting at the zeroth index.
label_id_offset = 1
score_threshold = 0.3
batch_size = 64
learning_rate = 0.01
epochs = 2
num_batches = round(len(glob.glob(train_augmented_fp + '/*')) / batch_size) # AKA "Iterations"
num_classes = 4

print("Done")

Done


In [12]:
train_file_list = glob.glob(train_augmented_fp + '/*')
val_file_list = glob.glob(val_augmented_fp + '/*')

column_names = ["Label", "X", "Y", "x", "y"]
train_csv_dataset = pd.DataFrame(columns = column_names)
val_csv_dataset = pd.DataFrame(columns = column_names)

with tqdm(total = len(train_file_list), unit="file") as pbar:
    for filename in train_file_list:
        _, label, bounding_box = GEORGE.get_file_and_info(filename)
        train_csv_dataset = train_csv_dataset.append({"Label": label[0]['name'], "X": bounding_box[0][0], "Y": bounding_box[0][1], "x": bounding_box[0][2], "y": bounding_box[0][3]}, ignore_index = True)
        pbar.update(1)

filepath = Path('train_dataset.csv')
train_csv_dataset.to_csv(filepath)

with tqdm(total = len(val_file_list), unit="file") as pbar:
    for filename in val_file_list:
        _, label, bounding_box = GEORGE.get_file_and_info(filename)
        val_csv_dataset = val_csv_dataset.append({"Label": label[0]['name'], "X": bounding_box[0][0], "Y": bounding_box[0][1], "x": bounding_box[0][2], "y": bounding_box[0][3]}, ignore_index = True)
        #pbar.set_postfix(loss=total_loss.numpy())
        pbar.update(1)
print(val_csv_dataset)

filepath = Path('val_dataset.csv')
val_csv_dataset.to_csv(filepath)

print("Done")

100%|███████████████████████████████████████████████████████████████████████████| 1001/1001 [00:05<00:00, 179.83file/s]

        Label         X         Y         x         y
0     regular  0.001563  0.639063  0.131250  0.731250
1     regular  0.004688  0.706250  0.135937  0.826563
2     regular  0.006250  0.090625  0.512500  0.581250
3     regular  0.007812  0.501562  0.282813  0.989062
4     regular  0.009375  0.209375  0.518750  0.696875
...       ...       ...       ...       ...       ...
996     wasps  0.575000  0.254687  0.812500  0.726562
997     wasps  0.629687  0.065625  0.945312  0.571875
998     wasps  0.639063  0.173438  0.892187  0.651563
999     wasps  0.673437  0.421875  0.985937  0.925000
1000    wasps  0.679688  0.217187  0.970312  0.710938

[1001 rows x 5 columns]
Done
